In [40]:
from pykafka import KafkaClient
import time

In [41]:
client = KafkaClient(hosts="127.0.0.1:9092")

In [42]:
topic = client.topics['geostream']
producer = topic.get_sync_producer()

In [43]:
with topic.get_sync_producer() as producer:
    i = 0
    while True:
        i = i + 1
        producer.produce(('test message ' + str(i ** 2)).encode("ascii"))
        time.sleep(1)
        

KeyboardInterrupt: 

In [1]:
import requests
import time

In [21]:
msg = {
    "name": "messenger_2",
    "lat": 51.1,
    "lon": 13.3
}

In [22]:
while True:
    requests.post("http://127.0.0.1:8002/producer/geostream", json=msg)
    time.sleep(1)

KeyboardInterrupt: 

In [78]:
response = requests.post("http://127.0.0.1:8002/producer/geostream", json=msg)
response.content

b'{"name":"messenger_2","message_id":"messenger_2_4c6e4dee-87bb-4d12-bcc0-120a65b2a3a4","timestamp":"2020-03-05 16:36:59.412857","lat":51.1,"lon":13.3}'

In [12]:
from pydantic import BaseModel, StrictStr, confloat, validator
from datetime import datetime
import uuid

class ProducerMessage(BaseModel):
    name: StrictStr
    message_id: StrictStr = ""
    timestamp: StrictStr = str(datetime.utcnow())
    lat: confloat(gt=-90, lt=90)
    lon: confloat(gt=-180, lt=180)

    @validator("message_id", pre=True, always=True)
    def set_id_from_name_uuid(cls, v, values):
        if "name" in values:
            return f"{values['name']}_{uuid.uuid4()}"
        else:
            raise ValueError("name not set")


In [13]:
mes = ProducerMessage(**msg) 

In [14]:
mes.timestamp

'2020-03-05 16:27:19.726595'